In [1]:
from nltk import wsd
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
from spacy.cli import download
from spacy import load
import warnings
import json
import re
import polars as pl

nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('wordnet2022')
#!python -m spacy download en
nlp = load('en_core_web_sm')

[nltk_data] Downloading package omw-1.4 to /users/ujan/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to /users/ujan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package wordnet2022 to
[nltk_data]     /users/ujan/nltk_data...
[nltk_data]   Package wordnet2022 is already up-to-date!
span-marker is already registered. Overwriting pipeline for task span-marker...


In [5]:
X = 'The die is cast.'
Y = 'Roll the die to get a 6.'
Z = 'What is dead may never die.'

In [6]:
wn.synsets("die")

[Synset('die.n.01'),
 Synset('die.n.02'),
 Synset('die.n.03'),
 Synset('die.v.01'),
 Synset('die.v.02'),
 Synset('die.v.03'),
 Synset('fail.v.04'),
 Synset('die.v.05'),
 Synset('die.v.06'),
 Synset('die.v.07'),
 Synset('die.v.08'),
 Synset('die.v.09'),
 Synset('die.v.10'),
 Synset('die.v.11')]

In [7]:
# print all the definations of nouns
i =0
for syn in wn.synsets('die', pos=wn.NOUN):
    print("defination {0} : {1}".format(i, syn.definition()))
    i+=1

defination 0 : a small cube with 1 to 6 spots on the six faces; used in gambling to generate random numbers
defination 1 : a device used for shaping metal
defination 2 : a cutting tool that is fitted into a diestock and used for cutting male (external) screw threads on screws or bolts or pipes or rods


In [8]:
print(X)
print(wsd.lesk(X.split(), 'die'))
print(wsd.lesk(X.split(), 'die').definition())

The die is cast.
Synset('die.v.07')
cut or shape with a die


In [9]:
wn.synsets("prize")

[Synset('prize.n.01'),
 Synset('loot.n.01'),
 Synset('trophy.n.02'),
 Synset('prize.v.01'),
 Synset('pry.v.01'),
 Synset('respect.v.01'),
 Synset('choice.s.01')]

In [69]:
# print all the definations of nouns
i =0
for syn in wn.synsets('course', pos=wn.NOUN):
    print("defination {0} : {1}".format(i, syn.definition()))
    i+=1

defination 0 : education imparted in a series of lessons or meetings
defination 1 : a connected series of events or actions or developments
defination 2 : general line of orientation
defination 3 : a mode of action
defination 4 : a line or route along which something travels or moves
defination 5 : a body of students who are taught together
defination 6 : part of a meal served at one time
defination 7 : (construction) a layer of masonry
defination 8 : facility consisting of a circumscribed area of land or water laid out for a sport


In [71]:
# print all the definations of nouns
i =0
for syn in wn.synsets('draw', pos=wn.VERB):
    print("defination {0} : {1}".format(i, syn.definition()))
    i+=1

defination 0 : cause to move by pulling
defination 1 : get or derive
defination 2 : make a mark or lines on a surface
defination 3 : make, formulate, or derive in the mind
defination 4 : bring, take, or pull out of a container or from under a cover
defination 5 : represent by making a drawing of, as with a pencil, chalk, etc. on a surface
defination 6 : take liquid out of a container or well
defination 7 : give a description of
defination 8 : select or take in from a given group or region
defination 9 : elicit responses, such as objections, criticism, applause, etc.
defination 10 : suck in or take (air)
defination 11 : move or go steadily or gradually
defination 12 : remove (a commodity) from (a supply source)
defination 13 : choose at random
defination 14 : earn or achieve a base by being walked by the pitcher
defination 15 : bring or lead someone to a certain action or condition
defination 16 : cause to flow
defination 17 : write a legal document or paper
defination 18 : engage in dr

In [13]:
POS_MAP = {
    'VERB': wn.VERB,
    'NOUN': wn.NOUN,
    'PROPN': wn.NOUN
}


def lesk(doc, word):
    found = False
    for token in doc:
        if token.text == word:
            word = token
            found = True
            break
    if not found:
        raise ValueError(f'Word \"{word}\" does not appear in the document: {doc.text}.')
    pos = POS_MAP.get(word.pos_, False)
    if not pos:
        warnings.warn(f'POS tag for {word.text} not found in wordnet. Falling back to default Lesk behaviour.')
    args = [c.text for c in doc], word.text
    kwargs = dict(pos=pos)
    return wsd.lesk(*args, **kwargs)

In [16]:
doc = nlp('What is dead may never die.')
lesk(doc, 'die')

Synset('die.v.07')

In [17]:
lesk(doc, 'die').definition()

'cut or shape with a die'

In [18]:
lesk(nlp('I work at google.'), 'google').definition()

"a widely used search engine that uses text-matching techniques to find web pages that are important and relevant to a user's search"

In [19]:
lesk(nlp('I will google it.'), 'google').definition()

'search the internet (for information) using the Google search engine'

In [20]:
lesk(nlp('I hope we win!'), 'hope').definition()

'intend with some possibility of fulfilment'

In [23]:
with open('/users/ujan/Downloads/exact_matches_politics_42_10000_v2_latest.json') as f:
    pol_data = json.load(f)

In [53]:
prize_ids = pol_data['course'][1:][0]
prize_ids

['cx29aad',
 'dfn34if',
 'dho9j4b',
 'eri5wv8',
 'ct2h93v',
 'dedn867',
 'fs1c2dp',
 'g5fnns1',
 'dhh6o99',
 'dbmnncp',
 'cgk72ox',
 'fic4amk',
 'ci6s2dk',
 'cihkiev',
 'e33fq0a',
 'e33fq0a',
 'e33fq0a',
 'e33fq0a',
 'e33fq0a',
 'e33fq0a',
 'dg4iw5p',
 'cger8i9',
 'fzpyjy6',
 'fci1xv2',
 'fofbo3d',
 'dznko8q',
 'gh79bgx',
 'fmoka4x',
 'ef0sted',
 'fq5rg2l',
 'f5oysxv',
 'h07ql5b',
 'e68dugz',
 'd4elksb',
 'g7xz8bb',
 'fp37o38',
 'fj8zlex',
 'cli5zdx',
 'h31hy41',
 'cf69n26',
 'eodh4h0',
 'hgct5i2',
 'd6hcpdo',
 'hbhgxbl',
 'daa9zma',
 'e3o8cm2',
 'ep16pxp',
 'dpjrjcu',
 'f4ona9f',
 'el4fggb',
 'duxliwd',
 'cro7o9v',
 'gx17nhc',
 'fx1wlek',
 'gy06d12',
 'dvdxyvs',
 'dxgrvca',
 'f7dyatk',
 'daf9m1n',
 'g35976g',
 'd1ouezr',
 'evc8jlb',
 'e4m7lxx',
 'g7boje5',
 'd19onlp',
 'dht86g5',
 'daxu4g2',
 'cglpso8',
 'frwf2wf',
 'fe6fxoz',
 'gq74xfg',
 'e0fuk3n',
 'd9vzcgz',
 'efoatmm',
 'daf903z',
 'dudtq65',
 'e8x8qkr',
 'fz1o3ym',
 'd0e7xg0',
 'dp28x8x',
 'gfb5gvw',
 'd099gqx',
 'd6wb0gk',
 'er

In [54]:
data_df = pl.read_csv('/users/ujan/sports-language-in-politics/data/processed/politics_sample.csv').drop_nulls()
data_df.head()

id,created_utc,subreddit,category,super_category,body,author
str,i64,str,str,str,str,str
"""er1nbsy""",1560434110,"""AOC""","""politics_2019""","""politics""","""Lol. I was ask…","""Just_WoW_Thing…"
"""gq5rux6""",1615164573,"""AOC""","""politics_2019""","""politics""","""stfu Liberal.""","""gbsedillo20"""
"""gj3jztp""",1610531437,"""AOC""","""politics_2019""","""politics""",""" &gt;Fuck that…","""64590949354397…"
"""esrc8yq""",1562221646,"""AOC""","""politics_2019""","""politics""","""Cauliflower is…","""TobiKato"""
"""gyjpk7u""",1621327024,"""AOC""","""politics_2019""","""politics""","""If they build …","""the_lonely_gam…"


In [55]:
prize_comments = data_df.filter(pl.col('id').is_in(prize_ids))['body'].to_list()
prize_comments = [re.sub(r"[^a-zA-Z0-9]+", ' ', comment).lower() for comment in prize_comments]

In [57]:
prize_comments[0]

'that s exactly it without education 0 we re below average animals they run away earlier than us usually that s also one reason people mentionned about east asia numbers they had sars to learn and remember 1 so their reaction was clear and quick while the west took a month or more of swinging left and right until the conclusion drew itself on the board 0 of course epidemiologist and medical staff have some but they re not authorities and people barely respect authorities so here you go 1 interesting fractal effect society members memory is an immune system and covid 19 is our first flu shot '

In [62]:
prize_comments[3]

'of course you claim its that high your a trump supporter the polls are all over the place because right wing polls claim he is more popular then it is trump supporters are the scum of the earth on level with isis if trump is over 35 i would be surprised '

In [60]:
lesk(nlp('of course epidemiologist and medical staff have some but they re not authorities and people barely respect authorities'), 'course').definition()

'facility consisting of a circumscribed area of land or water laid out for a sport'

In [64]:
lesk(nlp('of course you claim its that high your a trump supporter the polls are all over the place because right wing polls claim he is more popular then it is trump supporters are the scum of the earth on level with isis if trump is over 35 i would be surprised '), 'course').definition()

'a body of students who are taught together'

In [72]:
from datasets import load_dataset

In [75]:
semcor = load_dataset('MarkChen1214/SemCor')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/20138 [00:00<?, ? examples/s]

In [76]:
semcor

DatasetDict({
    train: Dataset({
        features: ['ID', 'Word', 'Lemma', 'POS', 'Definition', 'Lemma_sentence', 'sentence', 'Lemma_tfidf', 'Lemma_tfidf_value'],
        num_rows: 20138
    })
})

In [78]:
semcor['train'][0]['sentence']

'Polyphosphates gave renewed life to soap products at a time when surfactants were a threat though expensive , and these same polyphosphates spelled the decline of soap usage when the synergism between polyphosphates and synthetic detergent actives was recognized and exploited .'

In [81]:
semcor['train'][10]['Definition']

['resisting weight or pressure',
 'the outer boundary of an artifact or a material layer constituting or resembling such a boundary',
 'a preparation used in cleaning something',
 'to consider or examine in speech or writing',
 'a subdivision of a written work; usually numbered and titled',
 'coming next after the twenty-seventh in position']

In [80]:
semcor['train'][0]['Word']

['Polyphosphates',
 'gave',
 'renewed',
 'life',
 'soap',
 'products',
 'time',
 'surfactants',
 'were',
 'threat',
 'expensive',
 'same',
 'polyphosphates',
 'spelled',
 'decline',
 'soap',
 'usage',
 'synergism',
 'polyphosphates',
 'synthetic',
 'detergent',
 'actives',
 'recognized',
 'exploited']

#### WSD with BERT MASK probs

In [2]:
from transformers import AutoTokenizer, BertForMaskedLM
import torch
import nltk
from nltk.corpus import wordnet
from itertools import chain

In [3]:
model_name = 'bert-base-uncased'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = BertForMaskedLM.from_pretrained(model_name)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
sports_words = []
for w in wn.all_synsets():
    if 'sports' in w.definition():
        sports_words.extend(w.lemma_names())

In [14]:
len(sports_words)

207

In [15]:
sports_words[:50]

['clean',
 'sporting',
 'sporty',
 'sportsmanlike',
 'sportive',
 'sporting',
 'unsportingly',
 'fumble',
 'muff',
 'pass',
 'toss',
 'flip',
 'place_kick',
 'place-kicking',
 'tag',
 'call',
 'hat_trick',
 'track_and_field',
 'water_sport',
 'aquatics',
 'sporting_life',
 'victory_celebration',
 'play',
 'assist',
 'stroke',
 'shot',
 'cut',
 'undercut',
 'drive',
 'forehand',
 'forehand_stroke',
 'forehand_shot',
 'forehand_drive',
 'serve',
 'service',
 'fault',
 'position',
 'technical_foul',
 'technical',
 'athletics',
 'replay',
 'instant_replay',
 'action_replay',
 'save',
 'rally',
 'exchange',
 'cheap_shot',
 'dirty_pool',
 'sporting_dog',
 'gun_dog']

In [16]:
count = 0
for word in sports_words:
    if word in sorted_token_probs:
        count += 1

count

89

In [22]:
TARGET = 'course'

comments = [
    'I will be at the course later',
    'Of course I will see you later'
]

token_dict = {}
for comment in comments:
    tokens = comment.split()
    tokens = ['[MASK]' if token == TARGET else token for token in tokens]
    masked_comment = ' '.join(tokens)
    inputs = tokenizer(masked_comment, return_tensors="pt", truncation=True)
    if inputs['input_ids'].shape[-1] >= 512:
        continue
    with torch.no_grad():
        logits = model(**inputs).logits
        # get mask token ids
        mask_token_ids = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
        # for each masked token (multiple biden mentions)
        for mask_token_id in mask_token_ids:
            # get corresponding logits
            mask_logits = logits[0, mask_token_id.item()]
            # softmax to get probs for each token in vocab
            mask_probs = torch.nn.functional.softmax(mask_logits, dim=0)
            # sort in descending order
            prob_values, token_ids = torch.sort(mask_probs, descending=True)
            # for each token in vocab
            for i in range(len(token_ids)):
                # decode to get string
                token = tokenizer.decode(token_ids[i])
                # prob value
                value = prob_values[i].item()
                
                if token in token_dict:
                    token_dict[token]['value'] += value
                    token_dict[token]['count'] += 1
                else:
                    token_dict[token] = {'value': value, 'count': 1}

    new_token_dict = {}
    for key, val in token_dict.items():
        new_token_dict[key] = val['value'] / val['count']

    sorted_token_probs = dict(sorted(new_token_dict.items(), key=lambda item: item[1], reverse=True))
    prob_value = 0
    for word in sports_words:
        if word in sorted_token_probs:
            prob_value += sorted_token_probs[word]
    print(prob_value)

0.029426940073832952
0.014713527005746851


In [18]:
TARGET = 'card'

comments = [
    'He was shown a red card for the tackle',
    'My credit card expired yesterday'
]

token_dict = {}
for comment in comments:
    tokens = comment.split()
    tokens = ['[MASK]' if token == TARGET else token for token in tokens]
    masked_comment = ' '.join(tokens)
    inputs = tokenizer(masked_comment, return_tensors="pt", truncation=True)
    if inputs['input_ids'].shape[-1] >= 512:
        continue
    with torch.no_grad():
        logits = model(**inputs).logits
        # get mask token ids
        mask_token_ids = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
        # for each masked token (multiple biden mentions)
        for mask_token_id in mask_token_ids:
            # get corresponding logits
            mask_logits = logits[0, mask_token_id.item()]
            # softmax to get probs for each token in vocab
            mask_probs = torch.nn.functional.softmax(mask_logits, dim=0)
            # sort in descending order
            prob_values, token_ids = torch.sort(mask_probs, descending=True)
            # for each token in vocab
            for i in range(len(token_ids)):
                # decode to get string
                token = tokenizer.decode(token_ids[i])
                # prob value
                value = prob_values[i].item()
                
                if token in token_dict:
                    token_dict[token]['value'] += value
                    token_dict[token]['count'] += 1
                else:
                    token_dict[token] = {'value': value, 'count': 1}

    new_token_dict = {}
    for key, val in token_dict.items():
        new_token_dict[key] = val['value'] / val['count']

    sorted_token_probs = dict(sorted(new_token_dict.items(), key=lambda item: item[1], reverse=True))
    prob_value = 0
    for word in sports_words:
        if word in sorted_token_probs:
            prob_value += sorted_token_probs[word]
    print(prob_value)

0.0005403707188477158
0.0002809585464422058


In [21]:
TARGET = 'fan'

comments = [
    'He was a life long fan of his local club',
    'His actions only served to fan the flames',
]

token_dict = {}
for comment in comments:
    tokens = comment.split()
    tokens = ['[MASK]' if token == TARGET else token for token in tokens]
    masked_comment = ' '.join(tokens)
    inputs = tokenizer(masked_comment, return_tensors="pt", truncation=True)
    if inputs['input_ids'].shape[-1] >= 512:
        continue
    with torch.no_grad():
        logits = model(**inputs).logits
        # get mask token ids
        mask_token_ids = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
        # for each masked token (multiple biden mentions)
        for mask_token_id in mask_token_ids:
            # get corresponding logits
            mask_logits = logits[0, mask_token_id.item()]
            # softmax to get probs for each token in vocab
            mask_probs = torch.nn.functional.softmax(mask_logits, dim=0)
            # sort in descending order
            prob_values, token_ids = torch.sort(mask_probs, descending=True)
            # for each token in vocab
            for i in range(len(token_ids)):
                # decode to get string
                token = tokenizer.decode(token_ids[i])
                # prob value
                value = prob_values[i].item()
                
                if token in token_dict:
                    token_dict[token]['value'] += value
                    token_dict[token]['count'] += 1
                else:
                    token_dict[token] = {'value': value, 'count': 1}

    new_token_dict = {}
    for key, val in token_dict.items():
        new_token_dict[key] = val['value'] / val['count']

    sorted_token_probs = dict(sorted(new_token_dict.items(), key=lambda item: item[1], reverse=True))
    prob_value = 0
    for word in sports_words:
        if word in sorted_token_probs:
            prob_value += sorted_token_probs[word]
    print(prob_value)

0.286751383745389
0.15239340089198766


##### 'context free' probs

In [10]:
TARGET = 'word'


# do this for important words #
comments = [
    '[PAD] word [PAD]',
    #'My credit card expired yesterday'
]

token_dict = {}
for comment in comments:
    tokens = comment.split()
    tokens = ['[MASK]' if token == TARGET else token for token in tokens]
    masked_comment = ' '.join(tokens)
    inputs = tokenizer(masked_comment, return_tensors="pt", truncation=True)
    if inputs['input_ids'].shape[-1] >= 512:
        continue
    with torch.no_grad():
        logits = model(**inputs).logits
        # get mask token ids
        mask_token_ids = (inputs.input_ids == tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
        # for each masked token (multiple biden mentions)
        for mask_token_id in mask_token_ids:
            # get corresponding logits
            mask_logits = logits[0, mask_token_id.item()]
            # softmax to get probs for each token in vocab
            mask_probs = torch.nn.functional.softmax(mask_logits, dim=0)
            # sort in descending order
            prob_values, token_ids = torch.sort(mask_probs, descending=True)
            # for each token in vocab
            for i in range(len(token_ids)):
                # decode to get string
                token = tokenizer.decode(token_ids[i])
                # prob value
                value = prob_values[i].item()
                
                if token in token_dict:
                    token_dict[token]['value'] += value
                    token_dict[token]['count'] += 1
                else:
                    token_dict[token] = {'value': value, 'count': 1}

    new_token_dict = {}
    for key, val in token_dict.items():
        new_token_dict[key] = val['value'] / val['count']

    sorted_token_probs = dict(sorted(new_token_dict.items(), key=lambda item: item[1], reverse=True))
    #print(sorted_token_probs)

In [9]:
# word

words = ['race', 'election', 'fans', 'voters']

for key, val in sorted_token_probs.items():
    if key in words:
        print(key, val)

election 1.6974567584870925e-10
race 9.876374718054137e-11
fans 6.21402374001434e-11
voters 1.1286636555918328e-11


In [11]:
# pad word pad

words = ['race', 'election', 'fans', 'voters']

for key, val in sorted_token_probs.items():
    if key in words:
        print(key, val)

voters 2.48651190304372e-06
election 1.96735300050932e-06
race 4.895503593616013e-07
fans 3.7076819126014016e-07
